In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [56]:
#example url
url='https://indeks.kompas.com/?site=tekno&date=2019-12-06'
url

'https://indeks.kompas.com/?site=tekno&date=2019-12-06'

In [57]:
#send request to the url
req=requests.get(url)
soup=BeautifulSoup(req.text, 'lxml') #generate text as a BeautifulSoup Object 
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="index, follow" name="robots"/>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-PT7PC4');</script>
<!-- End Google Tag Manager -->
<title>Indeks Berita Tekno Halaman 1 - 6 Desember 2019 - Kompas.com</title>
<link href="//securepubads.g.doubleclick.net" rel="dns-prefetch"/>
<link href="//asset.kompas.com" rel="dns-prefetch"/>
<link href="//adsimg.kompas.com" rel="dns-prefetch"/>
<link href="//static.criteo.net" rel="dns-prefetch"/>
<link href="//bidder.criteo.com" rel="dns-prefetch"/>
<link href="//rtax.criteo.com" rel="dns-prefetch"/>
<link href="//tpc.googlesyndication.com" rel="dns-prefetch"/>
<link href="//cm.g.doubl

In [58]:
#find all the link article in those page
kumpulan_page=soup.find_all('div',class_='article__asset')
link_page=[kumpulan_page[i].a['href'] for i in range(len(kumpulan_page))]
for i in link_page: print(i)

https://tekno.kompas.com/read/2019/12/06/21241227/uu-perlindungan-data-pribadi-dulu-data-center-facebook-dan-google-kemudian
https://tekno.kompas.com/read/2019/12/06/20310037/spesifikasi-lengkap-dan-harga-asus-rog-phone-ii-di-indonesia
https://tekno.kompas.com/read/2019/12/06/19184737/iphone-11-rilis-di-indonesia-harga-iphone-xr-xs-dan-xs-max-turun
https://tekno.kompas.com/read/2019/12/06/18230097/ipad-generasi-7-resmi-masuk-indonesia-berapa-harganya-
https://tekno.kompas.com/read/2019/12/06/17410037/aturan-imei-bikin-peminat-iphone-11-di-indonesia-rela-menunggu-
https://tekno.kompas.com/read/2019/12/06/16110007/erajaya-klaim-antusiasme-indonesia-sambut-iphone-11-tinggi
https://tekno.kompas.com/read/2019/12/06/15450087/asus-jamin-stok-ponsel-rog-phone-2-di-indonesia-aman
https://tekno.kompas.com/read/2019/12/06/15120197/penyanyi-andien-ikut-komentari-performa-iphone-11
https://tekno.kompas.com/read/2019/12/06/14295787/ponsel-murah-nokia-23-meluncur-baterai-diklaim-tahan-2-hari
https://

In [59]:
#for example crawl one of page from all of articles url
i='https://tekno.kompas.com/read/2019/12/06/21241227/uu-perlindungan-data-pribadi-dulu-data-center-facebook-dan-google-kemudian'

In [60]:
item={}
i=i+'?page=all' #to see page all view without paging
req_berita=requests.get(i)
soup_berita=BeautifulSoup(req_berita.text, 'lxml')
soup_berita

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="94D7C64651DB01DACCF13907E3748127" name="msvalidate.01"/>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-PT7PC4');</script>
<!-- End Google Tag Manager -->
<title>UU Perlindungan Data Pribadi Dulu, "Data Center" Facebook dan Google Kemudian Halaman all - Kompas.com</title>
<link href="//securepubads.g.doubleclick.net" rel="dns-prefetch"/>
<link href="//asset.kompas.com" rel="dns-prefetch"/>
<link href="//adsimg.kompas.com" rel="dns-prefetch"/>
<link href="//static.criteo.net" rel="dns-prefetch"/>
<link href="//bidder.criteo.com" rel="dns-prefetch"/>
<link href="//rtax.criteo.com" rel="dns-prefetch"/>
<link href="//tpc.

In [61]:
#title
soup_berita.find_all('title')[0].text

'UU Perlindungan Data Pribadi Dulu, "Data Center" Facebook dan Google Kemudian Halaman all - Kompas.com'

In [62]:
#the date
tgl=soup_berita.find_all('div', {'class':'read__time'})[0].text
tgl.split(',')[0].split('-')[1].replace(' ',"")

'06/12/2019'

In [63]:
#article content
berita=soup_berita.find_all('div', {'class':'read__content'})[0]
berita.text.strip().replace('\n' , ' ')

'JAKARTA KOMPAS.com - Menteri Komunikasi dan Informatika Johnny Plate menyatakan Google dan Facebook berencana untuk membangun  data center di Indonesia. Namun sebelum rencana pusat server dua raksasa internet itu diwujudkan, Johnny menyebut bahwa Undang-Undang Perlindungan Data Pribadi ( UU PDP) harus terlebih dahulu disahkan. Johnny menegaskan, hal tersebut menjadi penting agar Indonesia dapat mengelola data dengan baik dan untuk memastikan agar data tidak disalahgunakan. "Ini tidak mudah, Kita telah bertransformasi dan integrasi dari dunia fisik ke dunia digital karenanya itu kita perlu meningkatkan keamanan data. Management data kita harus baik untuk keamanan negara dan kepentingan pertumbuhan ekonomi negara," ujar  Menkominfo Johnny. Ia mengatakan bahwa negara-negara lain sudah memiliki regulasi yang melindungi data pribadi, sementara Indonesia belum memilikinya. "Kita harus menyelesaikan UU PDP Indonesia. Saat ini bangsa-bangsa lain sudah punya, tetapi kita belum," tuturnya. Baca

In [64]:
#PHOTO URL
photo=soup_berita.find_all('div', {'class':'photo'})[0]
photo.img['data-src']

'https://asset.kompas.com/crops/s5A_X-nxLQhYAkhYASg0Aa0wF7Q=/0x0:1200x800/750x500/data/photo/2019/10/29/5db80d3f22be7.jpg'